# 第20章：Error Handling & Retry（错误处理与重试）

## 学习目标

本章将学习：
1. LangChain/LangGraph中的常见错误类型
2. 错误分类与处理策略
3. 重试中间件（Retry Middleware）
4. 工具错误处理
5. 模型错误处理
6. 结构化输出错误处理
7. 超时控制
8. 降级策略（Fallback）
9. 生产环境最佳实践

---

## 为什么需要错误处理？

### 生产环境的挑战

在生产环境中，各种错误不可避免：

| 错误来源 | 示例 | 影响 |
|---------|------|------|
| **网络问题** | API超时、连接失败 | 服务不可用 |
| **API限制** | 速率限制、配额超限 | 请求被拒绝 |
| **模型错误** | 解析失败、格式错误 | 输出不可用 |
| **工具失败** | 数据库连接失败、外部API错误 | 功能受限 |
| **用户输入** | 缺少必要信息、格式错误 | 无法完成任务 |

### 没有错误处理的后果

```python
# ❌ 没有错误处理
response = agent.invoke({"messages": [...]})
# → 一旦出错，整个应用崩溃
# → 用户看到500错误
# → 无法诊断问题
```

### 有错误处理的好处

```python
# ✅ 有错误处理
try:
    response = agent.invoke({"messages": [...]})
except Exception as e:
    # → 记录错误日志
    # → 返回友好提示
    # → 自动重试
    # → 优雅降级
```

---

## 核心概念：错误分类

LangChain/LangGraph将错误分为4类，每类有不同的处理策略：

### 错误分类表

| 错误类型 | 谁来修复 | 处理策略 | 适用场景 |
|---------|---------|---------|---------|
| **Transient errors**<br>（临时性错误） | 系统（自动） | Retry Policy<br>（重试策略） | 网络问题、速率限制<br>通常重试后可恢复 |
| **LLM-recoverable errors**<br>（LLM可恢复错误） | LLM | 将错误存入状态<br>让LLM看到并调整 | 工具失败、解析错误<br>LLM可以调整策略 |
| **User-fixable errors**<br>（用户可修复错误） | 人类 | Human-in-the-loop<br>暂停并请求输入 | 缺少信息、不明确指令<br>需要用户提供输入 |
| **Unexpected errors**<br>（未预期错误） | 开发者 | 让错误冒泡<br>记录日志 | 未知问题<br>需要开发者调试 |

### 关键原则

✅ **不同错误用不同策略** - 不要一刀切  
✅ **自动化优先** - 能自动处理的不要人工介入  
✅ **记录所有错误** - 便于排查和改进  
✅ **优雅降级** - 即使出错也要保证服务可用

In [2]:
# 环境配置
import os
import sys
import time

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. 工具错误处理（Tool Error Handling）

### 核心价值

工具执行时可能因为各种原因失败（如参数错误、外部API故障等），需要优雅地处理这些错误。

### 使用 `@wrap_tool_call` 中间件

LangChain提供 `@wrap_tool_call` 装饰器来自定义工具错误处理逻辑。

```python
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """处理工具执行错误"""
    try:
        return handler(request)
    except Exception as e:
        # 返回自定义错误消息给LLM
        return ToolMessage(
            content=f"工具执行失败：{str(e)}。请检查输入参数。",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model=model,
    tools=[my_tool],
    middleware=[handle_tool_errors]  # 应用中间件
)
```

### 何时使用工具错误处理

| 场景 | 是否使用 | 原因 |
|------|---------|------|
| **运行时输入错误** | ✅ 使用 | 如SQL语法错误、无效参数 |
| **网络故障** | ❌ 不用 | 应该用重试中间件 |
| **工具实现错误** | ❌ 不用 | 应该让错误冒泡，开发者修复 |
| **Schema不匹配** | ❌ 不用 | 框架自动处理 |

### 关键点

- ✅ 返回 `ToolMessage` 让LLM看到错误并调整
- ✅ 提供清晰的错误提示
- ✅ 只捕获运行时错误，不捕获实现错误

In [3]:
print("【演示1：工具错误处理】\n")

# 定义一个会出错的工具
@tool
def divide_numbers(a: float, b: float) -> float:
    """将两个数字相除"""
    return a / b  # 当b=0时会抛出ZeroDivisionError

# 定义错误处理中间件
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """捕获工具错误并返回友好提示"""
    try:
        result = handler(request)
        print(f"  ✓ 工具执行成功")
        return result
    except ZeroDivisionError:
        print(f"  ✗ 捕获到除零错误")
        return ToolMessage(
            content="错误：除数不能为0。请使用非零的除数重试。",
            tool_call_id=request.tool_call["id"]
        )
    except Exception as e:
        print(f"  ✗ 捕获到其他错误: {type(e).__name__}")
        return ToolMessage(
            content=f"工具执行失败：{str(e)}",
            tool_call_id=request.tool_call["id"]
        )

# 创建带错误处理的Agent
agent_with_error_handling = create_agent(
    model=model,
    tools=[divide_numbers],
    middleware=[handle_tool_errors],
    system_prompt="你是一个数学助手，可以进行除法运算。"
)

# 测试1：正常情况
print("=== 测试1：正常除法 ===")
response1 = agent_with_error_handling.invoke({
    "messages": [{"role": "user", "content": "计算 10 除以 2"}]
})
print(f"Agent回复: {response1['messages'][-1].content}\n")

# 测试2：除零错误
print("=== 测试2：除零错误（会被捕获） ===")
response2 = agent_with_error_handling.invoke({
    "messages": [{"role": "user", "content": "计算 10 除以 0"}]
})
print(f"Agent回复: {response2['messages'][-1].content}\n")

print("✓ 工具错误被优雅处理，Agent可以看到错误并调整")
print("✓ 用户不会看到崩溃，而是得到友好提示")


【演示1：工具错误处理】

=== 测试1：正常除法 ===
  ✓ 工具执行成功
Agent回复: 10 除以 2 等于 5。

=== 测试2：除零错误（会被捕获） ===
Agent回复: 除数不能为零，除以零是未定义的运算，无法计算。请提供一个非零的除数。

✓ 工具错误被优雅处理，Agent可以看到错误并调整
✓ 用户不会看到崩溃，而是得到友好提示


## 2. 工具重试中间件（Tool Retry Middleware）

### 核心价值

自动重试失败的工具调用，使用指数退避策略，适合处理**临时性错误**（如网络抖动、API速率限制）。

### ToolRetryMiddleware 参数

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `max_retries` | 最大重试次数 | 2 |
| `tools` | 要应用重试的工具列表（None=全部） | None |
| `retry_on` | 要重试的异常类型（tuple或callable） | 所有异常 |
| `on_failure` | 失败后的行为<br>- `'return_message'`: 返回错误消息<br>- `'raise'`: 抛出异常<br>- 自定义函数 | `'return_message'` |
| `backoff_factor` | 指数退避倍数 | 2.0 |
| `initial_delay` | 初始延迟（秒） | 1.0 |
| `max_delay` | 最大延迟（秒） | 60.0 |
| `jitter` | 是否添加随机抖动（±25%） | True |

### 重试延迟计算

```python
delay = initial_delay * (backoff_factor ** retry_number)
delay = min(delay, max_delay)  # 不超过max_delay

# 示例：initial_delay=1.0, backoff_factor=2.0
# 第1次重试: 1.0秒
# 第2次重试: 2.0秒
# 第3次重试: 4.0秒
```

### 使用场景

| 场景 | 是否使用 | 原因 |
|------|---------|------|
| **网络超时** | ✅ 推荐 | 临时性问题，重试可恢复 |
| **API速率限制** | ✅ 推荐 | 等待后可恢复 |
| **外部服务暂时不可用** | ✅ 推荐 | 503错误，重试可恢复 |
| **参数错误** | ❌ 不推荐 | 重试无意义，应该用错误处理 |
| **权限错误** | ❌ 不推荐 | 重试无法解决 |

In [5]:
print("【演示2：工具重试中间件】\n")

from langchain.agents.middleware import ToolRetryMiddleware

# 模拟一个不稳定的工具（前2次失败，第3次成功）
attempt_count = 0

@tool
def unstable_api_call(query: str) -> str:
    """调用一个不稳定的外部API"""
    global attempt_count
    attempt_count += 1
    
    print(f"  [尝试 {attempt_count}] 调用API...")
    
    if attempt_count < 3:
        # 模拟临时性网络错误
        print(f"  ✗ 模拟网络错误，第{attempt_count}次失败")
        raise ConnectionError("网络连接失败（临时性错误）")
    
    # 第3次成功
    return f"API返回结果：关于'{query}'的信息"

# 创建带重试的Agent
agent_with_retry = create_agent(
    model=model,
    tools=[unstable_api_call],
    middleware=[
        ToolRetryMiddleware(
            max_retries=3,           # 最多重试3次
            initial_delay=0.5,       # 初始延迟0.5秒
            backoff_factor=2.0,      # 指数退避
            jitter=False,            # 演示时关闭抖动
            retry_on=(ConnectionError,),  # 只重试ConnectionError
        )
    ],
    system_prompt="你是一个助手，可以调用外部API获取信息。"
)

print("=== 测试：调用不稳定的API ===")
start = time.time()

# 重置计数器
attempt_count = 0

response = agent_with_retry.invoke({
    "messages": [{"role": "user", "content": "查询Python的信息"}]
})

elapsed = time.time() - start

print(f"\nAgent回复: {response['messages'][-1].content}")
print(f"总耗时: {elapsed:.2f}秒")

【演示2：工具重试中间件】

=== 测试：调用不稳定的API ===
  [尝试 1] 调用API...
  ✗ 模拟网络错误，第1次失败
  [尝试 2] 调用API...
  ✗ 模拟网络错误，第2次失败
  [尝试 3] 调用API...

Agent回复: Python是一种广泛使用的高级编程语言，具有简洁易读的语法和强大的功能，适用于多种编程任务，如网页开发、数据分析、人工智能等。如果你需要更具体的Python信息或相关资源，请告诉我。
总耗时: 7.07秒


## 3. 模型重试中间件（Model Retry Middleware）

### 核心价值

自动重试失败的模型调用（LLM API），适合处理模型API的临时性错误。

### ModelRetryMiddleware 参数

与 `ToolRetryMiddleware` 参数类似：

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `max_retries` | 最大重试次数 | 2 |
| `retry_on` | 要重试的异常类型 | 所有异常 |
| `on_failure` | 失败后的行为<br>- `'continue'`: 返回AIMessage（默认）<br>- `'error'`: 抛出异常 | `'continue'` |
| `backoff_factor` | 指数退避倍数 | 2.0 |
| `initial_delay` | 初始延迟（秒） | 1.0 |
| `max_delay` | 最大延迟（秒） | 60.0 |
| `jitter` | 是否添加随机抖动 | True |

### 典型使用场景

```python
from langchain.agents.middleware import ModelRetryMiddleware

# 场景1：基础重试（默认设置）
agent = create_agent(
    model=model,
    tools=[...],
    middleware=[ModelRetryMiddleware()]
)

# 场景2：只重试特定错误
agent = create_agent(
    model=model,
    tools=[...],
    middleware=[
        ModelRetryMiddleware(
            max_retries=4,
            retry_on=(TimeoutError, ConnectionError),
            backoff_factor=1.5,
        )
    ]
)

# 场景3：自定义错误过滤
def should_retry(error: Exception) -> bool:
    # 只重试速率限制错误
    if hasattr(error, 'status_code'):
        return error.status_code == 429  # Rate limit
    return False

agent = create_agent(
    model=model,
    tools=[...],
    middleware=[
        ModelRetryMiddleware(
            max_retries=3,
            retry_on=should_retry,
        )
    ]
)
```

### 关键区别

| 维度 | ToolRetryMiddleware | ModelRetryMiddleware |
|------|-------------------|---------------------|
| **重试对象** | 工具调用 | LLM模型调用 |
| **默认on_failure** | `'return_message'` | `'continue'` |
| **适用场景** | 外部API、数据库查询 | LLM API调用 |

In [6]:
print("【演示3：模型重试中间件】\n")

from langchain.agents.middleware import ModelRetryMiddleware

# 注意：这里我们只演示配置方式，不实际触发模型错误
# 因为实际触发需要模拟API故障，比较复杂

print("=== 配置示例1：基础模型重试 ===")
agent_model_retry = create_agent(
    model=model,
    tools=[],
    middleware=[
        ModelRetryMiddleware(
            max_retries=3,
            initial_delay=1.0,
            backoff_factor=2.0,
        )
    ],
    system_prompt="你是一个助手"
)
print("✓ 已配置模型重试中间件")
print("  - 最多重试3次")
print("  - 初始延迟1秒，指数退避")

print("\n=== 配置示例2：只重试特定错误 ===")

# 自定义错误过滤函数
def should_retry_model_error(error: Exception) -> bool:
    """只重试速率限制和超时错误"""
    error_name = type(error).__name__
    
    # 检查是否是速率限制错误（不同API提供商可能不同）
    if 'RateLimitError' in error_name:
        print(f"  → 检测到速率限制错误，将重试")
        return True
    
    # 检查是否是超时错误
    if 'Timeout' in error_name:
        print(f"  → 检测到超时错误，将重试")
        return True
    
    # 其他错误不重试
    print(f"  → 检测到 {error_name}，不重试")
    return False

agent_selective_retry = create_agent(
    model=model,
    tools=[],
    middleware=[
        ModelRetryMiddleware(
            max_retries=4,
            retry_on=should_retry_model_error,  # 自定义过滤
            on_failure='continue',  # 失败后继续，返回错误消息
        )
    ],
    system_prompt="你是一个助手"
)
print("✓ 已配置选择性模型重试")
print("  - 只重试速率限制和超时错误")
print("  - 其他错误直接返回\n")

print("💡 关键点：")
print("  1. ModelRetryMiddleware 专门处理LLM API调用失败")
print("  2. 适合处理临时性网络问题和速率限制")
print("  3. 可以自定义重试策略，避免不必要的重试")
print("  4. 生产环境建议配置，提高系统可靠性")

【演示3：模型重试中间件】

=== 配置示例1：基础模型重试 ===
✓ 已配置模型重试中间件
  - 最多重试3次
  - 初始延迟1秒，指数退避

=== 配置示例2：只重试特定错误 ===
✓ 已配置选择性模型重试
  - 只重试速率限制和超时错误
  - 其他错误直接返回

💡 关键点：
  1. ModelRetryMiddleware 专门处理LLM API调用失败
  2. 适合处理临时性网络问题和速率限制
  3. 可以自定义重试策略，避免不必要的重试
  4. 生产环境建议配置，提高系统可靠性


## 4. 结构化输出错误处理

### 核心价值

当使用 `response_format` 生成结构化输出时，模型可能生成不符合schema的数据。LangChain提供智能重试机制。

### handle_errors 参数

在 `ToolStrategy` 中使用 `handle_errors` 参数自定义错误处理：

| 配置方式 | 行为 | 示例 |
|---------|------|------|
| `True` (默认) | 使用默认错误模板重试 | `handle_errors=True` |
| `False` | 不重试，直接抛出异常 | `handle_errors=False` |
| 字符串 | 使用固定错误消息重试 | `handle_errors="格式错误，请重试"` |
| 异常类型 | 只重试指定异常 | `handle_errors=ValueError` |
| 异常元组 | 只重试指定的多个异常 | `handle_errors=(ValueError, TypeError)` |
| 自定义函数 | 根据异常类型返回不同消息 | `handle_errors=custom_handler` |

### 使用示例

```python
from langchain.agents.structured_output import ToolStrategy
from pydantic import BaseModel, Field

class ProductRating(BaseModel):
    rating: int = Field(description="评分1-5", ge=1, le=5)
    comment: str = Field(description="评论")

# 方式1：使用默认错误处理
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=True  # 默认
    )
)

# 方式2：自定义错误消息
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors="评分必须在1-5之间，评论不能为空。"
    )
)

# 方式3：只重试特定异常
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=ValueError  # 只重试ValueError
    )
)

# 方式4：自定义错误处理函数
from langchain.agents.structured_output import StructuredOutputValidationError

def custom_error_handler(error: Exception) -> str:
    if isinstance(error, StructuredOutputValidationError):
        return "格式验证失败，请确保rating是1-5的整数。"
    return f"错误：{str(error)}"

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=custom_error_handler
    )
)
```

### 关键点

- ✅ 默认会自动重试，提高成功率
- ✅ 自定义错误消息可以引导模型生成正确格式
- ✅ 只重试可恢复的错误，避免无限循环

In [18]:
print("【演示4：结构化输出错误处理】\n")

from langchain.agents.structured_output import ToolStrategy
from pydantic import BaseModel, Field

# 定义结构化输出schema（严格验证）
class StrictProductRating(BaseModel):
    """产品评分（严格验证）"""
    product_name: str = Field(description="产品名称")
    rating: int = Field(description="评分1-5", ge=1, le=5)
    price: float = Field(description="价格（美元）", gt=0)
    would_recommend: bool = Field(description="是否推荐")

# 自定义错误处理函数（记录重试）
retry_count = 0

def rating_error_handler(error: Exception) -> str:
    """记录并处理验证错误"""
    global retry_count
    retry_count += 1
    
    error_type = type(error).__name__
    print(f"  ⚠️  [重试 {retry_count}] 检测到错误: {error_type}")
    
    if 'Validation' in error_type or 'validation' in str(error).lower():
        return """格式验证失败！请确保：
- rating 必须是1-5的整数
- price 必须是大于0的数字
- would_recommend 必须是true或false
- 所有字段都必须填写
请重新生成正确格式的数据。"""
    else:
        return f"发生错误：{str(error)}。请按照schema要求重新生成。"

# 创建带错误处理的Agent
agent_structured = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=StrictProductRating,
        handle_errors=rating_error_handler  # 自定义错误处理
    ),
    system_prompt="你是一个产品评论助手。根据用户的描述，提取产品信息并生成评分。"
)

print("=== 测试1：正常情况 ===")
retry_count = 0
response1 = agent_structured.invoke({
    "messages": [{"role": "user", "content": "iPhone 15 Pro很不错，我给4分，价格999美元，推荐购买"}]
})

structured_output1 = response1.get('structured_response')
if structured_output1:
    print(f"\n✓ 生成成功（重试{retry_count}次）")
    print(f"  产品: {structured_output1.product_name}")
    print(f"  评分: {structured_output1.rating}/5")
    print(f"  价格: ${structured_output1.price}")
    print(f"  推荐: {'是' if structured_output1.would_recommend else '否'}")

print("\n=== 测试2：容易触发错误的场景 ===")
print("（使用模糊描述，更可能需要重试）\n")
retry_count = 0

# 使用更模糊的描述，增加出错概率
response2 = agent_structured.invoke({
    "messages": [{"role": "user", "content": "这个笔记本电脑挺好的，性价比不错"}]
})

structured_output2 = response2.get('structured_response')
if structured_output2:
    print(f"\n✓ 最终生成成功（总共重试{retry_count}次）")
    print(f"  产品: {structured_output2.product_name}")
    print(f"  评分: {structured_output2.rating}/5")
    print(f"  价格: ${structured_output2.price}")
    print(f"  推荐: {'是' if structured_output2.would_recommend else '否'}")

print("\n💡 关键点：")
print("  1. handle_errors 参数会在验证失败时自动重试")
print("  2. 自定义错误消息帮助LLM理解要求")
print("  3. 严格的schema + 模糊的输入 = 更可能触发重试")
print("  4. 重试是自动的，对用户透明")

【演示4：结构化输出错误处理】

=== 测试1：正常情况 ===

✓ 生成成功（重试0次）
  产品: iPhone 15 Pro
  评分: 4/5
  价格: $999.0
  推荐: 是

=== 测试2：容易触发错误的场景 ===
（使用模糊描述，更可能需要重试）

  ⚠️  [重试 1] 检测到错误: StructuredOutputValidationError

✓ 最终生成成功（总共重试1次）
  产品: 笔记本电脑
  评分: 4/5
  价格: $300.0
  推荐: 是

💡 关键点：
  1. handle_errors 参数会在验证失败时自动重试
  2. 自定义错误消息帮助LLM理解要求
  3. 严格的schema + 模糊的输入 = 更可能触发重试
  4. 重试是自动的，对用户透明


## 5. 超时控制（Timeout Control）

### 核心价值

防止请求无限期等待，确保系统响应性。

### 实现方式

#### 方式1：使用 `invoke` 的 `config` 参数

```python
from langchain.errors import TimeoutError

try:
    response = agent.invoke(
        {"messages": [...]},
        config={
            "timeout": 30.0  # 30秒超时
        }
    )
except TimeoutError:
    print("请求超时")
```

#### 方式2：在模型初始化时设置

```python
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini",
    timeout=30.0,  # 30秒超时
    max_retries=2,  # 最多重试2次
)
```

#### 方式3：使用Python的 `asyncio.wait_for`（异步场景）

```python
import asyncio

try:
    response = await asyncio.wait_for(
        agent.ainvoke({"messages": [...]}),
        timeout=30.0
    )
except asyncio.TimeoutError:
    print("请求超时")
```

### 推荐超时时间

| 场景 | 推荐超时 | 原因 |
|------|---------|------|
| **简单问答** | 10-30秒 | 通常几秒内完成 |
| **复杂推理** | 60-120秒 | 需要更多思考时间 |
| **工具调用** | 30-60秒 | 取决于工具执行时间 |
| **批量处理** | 300秒+ | 需要处理多个任务 |
| **流式输出** | 不设置或很长 | 逐步返回，不需要严格超时 |

### 超时处理策略

```python
# ✅ 好的做法：超时后降级
try:
    response = agent.invoke(
        {"messages": [...]},
        config={"timeout": 30.0}
    )
except TimeoutError:
    # 降级：返回默认响应
    response = {"messages": [{"role": "assistant", "content": "抱歉，处理超时，请稍后重试。"}]}

# ❌ 坏的做法：超时后崩溃
response = agent.invoke({"messages": [...]}, config={"timeout": 30.0})
# 没有处理超时异常，应用崩溃
```


In [19]:
print("【演示5：超时控制】\n")

# 创建一个简单的Agent
simple_agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个助手"
)

print("=== 测试1：正常请求（无超时） ===")
try:
    start = time.time()
    response = simple_agent.invoke({
        "messages": [{"role": "user", "content": "用一句话介绍Python"}]
    })
    elapsed = time.time() - start
    print(f"✓ 请求成功，耗时: {elapsed:.2f}秒")
    print(f"  回复: {response['messages'][-1].content[:50]}...")
except Exception as e:
    print(f"✗ 请求失败: {e}")

print("\n=== 测试2：设置超时（演示配置） ===")
print("注意：实际超时需要请求真正超时才能触发")
print("这里只演示配置方式\n")

# 方式1：在invoke时设置超时
print("方式1：在invoke时设置超时")
print("""
try:
    response = agent.invoke(
        {"messages": [...]},
        config={"timeout": 30.0}  # 30秒超时
    )
except TimeoutError:
    print("请求超时，执行降级策略")
""")

# 方式2：在模型初始化时设置
print("\n方式2：在模型初始化时设置超时")
model_with_timeout = ChatOpenAI(
    model="gpt-4.1-mini",
    timeout=30.0,  # 30秒超时
    max_retries=2,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)
print("✓ 已创建带超时配置的模型")
print("  - 超时: 30秒")
print("  - 最多重试: 2次")

print("\n💡 超时控制的关键点：")
print("  1. 防止请求无限期等待")
print("  2. 根据场景设置合理的超时时间")
print("  3. 超时后应该有降级策略，而不是直接崩溃")
print("  4. 生产环境必须设置超时")


【演示5：超时控制】

=== 测试1：正常请求（无超时） ===
✓ 请求成功，耗时: 2.03秒
  回复: Python是一种简洁易学、功能强大的高级编程语言，广泛应用于数据分析、人工智能、网页开发等领域。...

=== 测试2：设置超时（演示配置） ===
注意：实际超时需要请求真正超时才能触发
这里只演示配置方式

方式1：在invoke时设置超时

try:
    response = agent.invoke(
        {"messages": [...]},
        config={"timeout": 30.0}  # 30秒超时
    )
except TimeoutError:
    print("请求超时，执行降级策略")


方式2：在模型初始化时设置超时
✓ 已创建带超时配置的模型
  - 超时: 30秒
  - 最多重试: 2次

💡 超时控制的关键点：
  1. 防止请求无限期等待
  2. 根据场景设置合理的超时时间
  3. 超时后应该有降级策略，而不是直接崩溃
  4. 生产环境必须设置超时


## 6. 降级策略（Fallback）

### 核心价值

当主要方法失败时，提供备用方案，确保服务可用性。

### 常见降级策略

#### 1. 模型降级

主模型失败时，切换到备用模型：

```python
from langchain_openai import ChatOpenAI

# 主模型：GPT-4
primary_model = ChatOpenAI(model="gpt-4", timeout=30)

# 备用模型：GPT-3.5
fallback_model = ChatOpenAI(model="gpt-3.5-turbo", timeout=30)

# 使用fallback
model_with_fallback = primary_model.with_fallbacks([fallback_model])

# 如果GPT-4失败，自动切换到GPT-3.5
response = model_with_fallback.invoke("问题")
```

#### 2. 功能降级

复杂功能失败时，切换到简单功能：

```python
try:
    # 尝试使用Agent（复杂）
    response = agent.invoke({"messages": [...]})
except Exception:
    # 降级：直接调用模型（简单）
    response = model.invoke("问题")
```

#### 3. 缓存降级

API失败时，返回缓存结果：

```python
cache = {}

def get_response_with_cache(query):
    try:
        # 尝试调用API
        response = agent.invoke({"messages": [{"role": "user", "content": query}]})
        # 缓存结果
        cache[query] = response
        return response
    except Exception:
        # 降级：返回缓存
        if query in cache:
            return cache[query]
        else:
            return {"messages": [{"role": "assistant", "content": "服务暂时不可用"}]}
```

#### 4. 默认响应降级

所有方法都失败时，返回友好的默认响应：

```python
DEFAULT_RESPONSE = {
    "messages": [{
        "role": "assistant",
        "content": "抱歉，服务暂时不可用，请稍后重试。"
    }]
}

try:
    response = agent.invoke({"messages": [...]})
except Exception as e:
    logging.error(f"Agent failed: {e}")
    response = DEFAULT_RESPONSE
```

### 降级策略选择

| 场景 | 推荐策略 | 原因 |
|------|---------|------|
| **模型API故障** | 模型降级 | 切换到备用模型 |
| **工具调用失败** | 功能降级 | 跳过工具，直接回答 |
| **网络问题** | 缓存降级 | 返回历史结果 |
| **完全不可用** | 默认响应 | 保证服务不崩溃 |


In [21]:
print("【演示6：降级策略】\n")

# 演示1：模型降级（使用with_fallbacks）
print("=== 演示1：模型降级 ===")

# 注意：这里演示配置方式，实际使用时需要有效的API密钥
print("配置示例：")
print("""
# 主模型
primary_model = ChatOpenAI(model="gpt-4", timeout=30)

# 备用模型
fallback_model = ChatOpenAI(model="gpt-3.5-turbo", timeout=30)

# 组合
model_with_fallback = primary_model.with_fallbacks([fallback_model])

# 使用（自动降级）
response = model_with_fallback.invoke("问题")
""")
print("✓ 如果主模型失败，自动切换到备用模型\n")

# 演示2：功能降级
print("=== 演示2：功能降级 ===")

@tool
def complex_tool(query: str) -> str:
    """一个可能失败的复杂工具"""
    # 模拟失败
    raise RuntimeError("工具暂时不可用")

agent_complex = create_agent(
    model=model,
    tools=[complex_tool],
    system_prompt="你是一个助手"
)

def get_response_with_fallback(query: str):
    """带降级的响应函数"""
    try:
        # 尝试使用Agent（可能调用工具）
        print("  → 尝试使用Agent...")
        response = agent_complex.invoke({
            "messages": [{"role": "user", "content": query}]
        })
        return response
    except Exception as e:
        # 降级：直接使用模型（不调用工具）
        print(f"  → Agent失败，降级到直接模型调用")
        response = model.invoke(query)
        return {"messages": [{"role": "assistant", "content": response.content}]}

result = get_response_with_fallback("介绍Python")

# 统一处理：兼容AIMessage对象和字典
last_msg = result['messages'][-1]
if hasattr(last_msg, 'content'):
    # AIMessage对象，使用属性访问
    content = last_msg.content
else:
    # 字典，使用字典访问
    content = last_msg['content']

print(f"最终回复: {content[:50]}...")
print("✓ 复杂功能失败后，降级到简单功能\n")

# 演示3：缓存降级
print("=== 演示3：缓存降级 ===")

response_cache = {}

def get_with_cache(query: str, use_cache: bool = False):
    """带缓存的响应函数"""
    
    # 检查缓存
    if query in response_cache:
        print(f"  → 缓存中存在结果")
    
    if use_cache and query in response_cache:
        print(f"  → 模拟API失败，使用缓存")
        return response_cache[query]
    
    try:
        # 正常调用API
        print(f"  → 调用API...")
        response = model.invoke(query)
        result = {"content": response.content}
        # 存入缓存
        response_cache[query] = result
        return result
    except Exception:
        # 降级：使用缓存
        if query in response_cache:
            print(f"  → API失败，使用缓存")
            return response_cache[query]
        else:
            return {"content": "服务暂时不可用"}

# 第一次调用（建立缓存）
print("第一次调用：")
result1 = get_with_cache("什么是LangChain？")
print(f"回复: {result1['content'][:50]}...\n")

# 第二次调用（使用缓存）
print("第二次调用（模拟API失败）：")
result2 = get_with_cache("什么是LangChain？", use_cache=True)
print(f"回复: {result2['content'][:50]}...")
print("✓ API失败时，返回缓存结果\n")

print("💡 降级策略的关键：")
print("  1. 确保服务永远不会完全不可用")
print("  2. 根据场景选择合适的降级方案")
print("  3. 记录降级事件，便于监控和改进")
print("  4. 降级应该是透明的，用户体验影响最小")

【演示6：降级策略】

=== 演示1：模型降级 ===
配置示例：

# 主模型
primary_model = ChatOpenAI(model="gpt-4", timeout=30)

# 备用模型
fallback_model = ChatOpenAI(model="gpt-3.5-turbo", timeout=30)

# 组合
model_with_fallback = primary_model.with_fallbacks([fallback_model])

# 使用（自动降级）
response = model_with_fallback.invoke("问题")

✓ 如果主模型失败，自动切换到备用模型

=== 演示2：功能降级 ===
  → 尝试使用Agent...
最终回复: Python是一种高级编程语言，由Guido van Rossum于1991年首次发布。它以简洁易读...
✓ 复杂功能失败后，降级到简单功能

=== 演示3：缓存降级 ===
第一次调用：
  → 调用API...
回复: LangChain 是一个用于构建基于大型语言模型（LLM）的应用程序的开源框架。它提供了一套工具和...

第二次调用（模拟API失败）：
  → 缓存中存在结果
  → 模拟API失败，使用缓存
回复: LangChain 是一个用于构建基于大型语言模型（LLM）的应用程序的开源框架。它提供了一套工具和...
✓ API失败时，返回缓存结果

💡 降级策略的关键：
  1. 确保服务永远不会完全不可用
  2. 根据场景选择合适的降级方案
  3. 记录降级事件，便于监控和改进
  4. 降级应该是透明的，用户体验影响最小


## 7. 生产环境最佳实践

### 完整的错误处理架构

生产环境需要综合使用多种错误处理策略：

**核心组件**：
1. 日志记录 - 记录所有错误
2. 重试机制 - 自动重试临时性错误
3. 错误处理 - 捕获并转换错误
4. 超时控制 - 防止无限等待
5. 降级策略 - 确保服务可用

### 错误处理检查清单

#### 开发阶段
- 为所有工具添加错误处理
- 配置合适的重试策略
- 设置超时时间
- 添加日志记录
- 编写单元测试（包括错误场景）

#### 测试阶段
- 测试网络故障场景
- 测试API速率限制
- 测试超时场景
- 测试工具失败场景
- 测试并发压力

#### 生产阶段
- 监控错误率
- 监控重试次数
- 监控超时频率
- 设置告警阈值
- 定期review错误日志

### 监控指标

| 指标 | 说明 | 告警阈值 |
|------|------|---------|
| 错误率 | 失败请求 / 总请求 | > 5% |
| 重试率 | 重试次数 / 总请求 | > 20% |
| 超时率 | 超时请求 / 总请求 | > 10% |
| 降级率 | 降级次数 / 总请求 | > 15% |
| 平均响应时间 | 所有请求的平均耗时 | > 10秒 |

In [24]:
print("【演示7：生产级错误处理架构】\n")

import logging

# 1. 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("=== 构建生产级Agent ===\n")

# 2. 定义工具
@tool
def production_tool(query: str) -> str:
    """生产环境工具"""
    logger.info(f"Tool called with query: {query}")
    return f"处理结果：{query}"

# 3. 工具错误处理中间件
@wrap_tool_call
def production_error_handler(request, handler):
    """生产级错误处理"""
    try:
        result = handler(request)
        logger.info("Tool execution successful")
        return result
    except Exception as e:
        logger.error(f"Tool execution failed: {type(e).__name__} - {str(e)}")
        return ToolMessage(
            content=f"工具执行失败：{str(e)}。请重试或联系支持。",
            tool_call_id=request.tool_call["id"]
        )

# 4. 创建生产级Agent
production_agent = create_agent(
    model=model,
    tools=[production_tool],
    middleware=[
        # 工具重试（处理临时性错误）
        ToolRetryMiddleware(
            max_retries=2,
            initial_delay=1.0,
            backoff_factor=2.0,
            retry_on=(ConnectionError, TimeoutError),
        ),
        # 模型重试
        ModelRetryMiddleware(
            max_retries=2,
            initial_delay=1.0,
        ),
        # 工具错误处理
        production_error_handler,
    ],
    system_prompt="你是一个生产环境助手"
)

print("✓ 已创建生产级Agent")
print("  - 工具重试：最多2次")
print("  - 模型重试：最多2次")
print("  - 错误处理：自定义中间件")
print("  - 日志记录：已启用\n")

# 5. 生产级调用函数
def safe_invoke(agent, messages, timeout=30):
    """带完整错误处理的调用函数"""
    try:
        logger.info("Starting request")
        response = agent.invoke(
            {"messages": messages},
            config={"timeout": timeout}
        )
        logger.info("Request completed successfully")
        return {"status": "success", "response": response}
    
    except TimeoutError:
        logger.error("Request timeout")
        return {
            "status": "timeout",
            "response": {
                "messages": [{
                    "role": "assistant",
                    "content": "处理超时，请稍后重试。"
                }]
            }
        }
    
    except Exception as e:
        logger.error(f"Request failed: {type(e).__name__} - {str(e)}")
        return {
            "status": "error",
            "response": {
                "messages": [{
                    "role": "assistant",
                    "content": "服务暂时不可用，我们正在处理。请稍后重试。"
                }]
            }
        }

# 6. 测试生产级调用
print("=== 测试生产级调用 ===\n")

result = safe_invoke(
    production_agent,
    [{"role": "user", "content": "测试生产环境"}],
    timeout=30
)

print(f"状态: {result['status']}")

# 统一处理：兼容AIMessage对象和字典
last_msg = result['response']['messages'][-1]
if hasattr(last_msg, 'content'):
    # AIMessage对象，使用属性访问
    content = last_msg.content
else:
    # 字典，使用字典访问
    content = last_msg['content']

print(f"回复: {content[:50]}...\n")

print("💡 生产环境关键点：")
print("  1. 多层防护：重试 + 错误处理 + 超时 + 降级")
print("  2. 完整日志：记录所有关键事件")
print("  3. 优雅降级：确保服务永不崩溃")
print("  4. 监控告警：及时发现和处理问题")
print("  5. 用户友好：提供清晰的错误提示")

2025-12-23 23:16:21,245 - INFO - Starting request


【演示7：生产级错误处理架构】

=== 构建生产级Agent ===

✓ 已创建生产级Agent
  - 工具重试：最多2次
  - 模型重试：最多2次
  - 错误处理：自定义中间件
  - 日志记录：已启用

=== 测试生产级调用 ===



2025-12-23 23:16:23,212 - INFO - HTTP Request: POST https://new.gptgod.cloud/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-23 23:16:23,217 - INFO - Tool called with query: 测试生产环境
2025-12-23 23:16:23,218 - INFO - Tool execution successful
2025-12-23 23:16:26,042 - INFO - HTTP Request: POST https://new.gptgod.cloud/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-23 23:16:26,047 - INFO - Request completed successfully


状态: success
回复: 生产环境测试已处理，结果显示测试成功。如需进一步操作，请告知。...

💡 生产环境关键点：
  1. 多层防护：重试 + 错误处理 + 超时 + 降级
  2. 完整日志：记录所有关键事件
  3. 优雅降级：确保服务永不崩溃
  4. 监控告警：及时发现和处理问题
  5. 用户友好：提供清晰的错误提示


## 8. 总结与最佳实践

### 核心要点

#### 1. 错误分类决定处理策略

| 错误类型 | 处理方式 | 工具/方法 |
|---------|---------|----------|
| 临时性错误 | 自动重试 | `ToolRetryMiddleware`, `ModelRetryMiddleware` |
| LLM可恢复错误 | 返回错误给LLM | `@wrap_tool_call` 返回 `ToolMessage` |
| 用户可修复错误 | Human-in-the-loop | `interrupt()` |
| 未预期错误 | 记录日志并降级 | `logging` + 默认响应 |

#### 2. 重试策略配置

```python
# 工具重试
ToolRetryMiddleware(
    max_retries=3,              # 最多重试3次
    initial_delay=1.0,          # 初始延迟1秒
    backoff_factor=2.0,         # 指数退避（1s, 2s, 4s）
    retry_on=(ConnectionError,), # 只重试特定错误
    on_failure='return_message' # 失败后返回消息
)

# 模型重试
ModelRetryMiddleware(
    max_retries=2,
    initial_delay=1.0,
    on_failure='continue'  # 失败后继续（返回AIMessage）
)
```

#### 3. 错误处理优先级

1. **重试** - 自动处理临时性错误
2. **错误转换** - 将错误转换为LLM可理解的消息
3. **超时控制** - 防止无限等待
4. **降级** - 确保服务可用
5. **日志** - 记录所有错误事件

---

### 最佳实践

#### ✅ 推荐做法

1. **分层错误处理**
   - 工具层：`@wrap_tool_call` 捕获运行时错误
   - 重试层：`ToolRetryMiddleware` 处理临时性错误
   - 应用层：`try-except` 处理所有未捕获错误

2. **合理的重试配置**
   - 网络错误：重试3-5次，指数退避
   - API速率限制：重试2-3次，更长的延迟
   - 参数错误：不重试，直接返回错误消息

3. **完整的日志记录**
   - 记录错误类型、时间、上下文
   - 使用结构化日志（JSON格式）
   - 设置不同的日志级别（INFO, WARNING, ERROR）

4. **优雅的降级策略**
   - 模型降级：GPT-4 → GPT-3.5
   - 功能降级：Agent → 直接模型调用
   - 缓存降级：返回历史结果
   - 默认响应：友好的错误提示

5. **设置超时时间**
   - 所有外部调用都应该有超时
   - 根据场景设置合理的超时值
   - 超时后应该有降级策略

#### ❌ 避免的陷阱

1. **过度重试**
   - ❌ 对所有错误都重试
   - ✅ 只重试临时性错误

2. **忽略错误**
   - ❌ 空的 `except: pass`
   - ✅ 记录日志并返回友好提示

3. **重试无限循环**
   - ❌ 没有设置 `max_retries`
   - ✅ 设置合理的重试上限

4. **没有超时控制**
   - ❌ 请求可能无限期等待
   - ✅ 所有请求都设置超时

5. **错误信息不友好**
   - ❌ 直接返回技术错误信息
   - ✅ 返回用户可理解的提示

---

### 常见错误处理模式

#### 模式1：网络错误

```python
# 使用重试中间件
ToolRetryMiddleware(
    retry_on=(ConnectionError, TimeoutError),
    max_retries=3,
    backoff_factor=2.0
)
```

#### 模式2：工具参数错误

```python
# 使用错误处理中间件
@wrap_tool_call
def handle_errors(request, handler):
    try:
        return handler(request)
    except ValueError as e:
        return ToolMessage(
            content=f"参数错误：{str(e)}",
            tool_call_id=request.tool_call["id"]
        )
```

#### 模式3：结构化输出错误

```python
# 使用handle_errors参数
response_format=ToolStrategy(
    schema=MySchema,
    handle_errors="格式错误，请重新生成"
)
```

#### 模式4：完整的生产级处理

```python
def production_invoke(agent, messages):
    try:
        # 尝试调用
        return agent.invoke(
            {"messages": messages},
            config={"timeout": 30}
        )
    except TimeoutError:
        # 超时降级
        logger.error("Timeout")
        return default_response
    except Exception as e:
        # 完全失败降级
        logger.error(f"Failed: {e}")
        return fallback_response
```

---

## 下一步学习

完成错误处理学习后，建议学习：

1. **第21章：Evaluation & Testing** - 评估和测试
2. **第22章：LangSmith Integration** - 监控和调试
3. **综合项目** - 构建生产级应用

---

## 关键收获

✅ **错误分类是关键** - 不同错误用不同策略

✅ **重试处理临时性错误** - `ToolRetryMiddleware`, `ModelRetryMiddleware`

✅ **错误转换让LLM自我修复** - `@wrap_tool_call` 返回 `ToolMessage`

✅ **超时防止无限等待** - 所有请求都应设置超时

✅ **降级确保服务可用** - 多层降级策略

✅ **日志便于排查问题** - 记录所有错误事件

✅ **生产环境需要多层防护** - 重试 + 错误处理 + 超时 + 降级 + 日志

✅ **用户体验优先** - 提供友好的错误提示，而不是技术错误